In [37]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [164]:
dataset_read= pd.read_csv('PIB_Ecuador.csv',sep=',')
dataset_read = dataset_read[dataset_read['Country Name']=='Ecuador']
dataset_read.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
64,Ecuador,ECU,PIB per cápita (UMN actual),NY.GDP.PCAP.CN,379.37,343.79,315.54,368.45,440.35,454.98,...,"5,200.56","5,682.05","6,056.33","6,377.09","6,124.49","6,060.09","6,213.50","6,295.94","6,183.82",nan


In [165]:
dataset_read = dataset_read[dataset_read['Country Name']=='Ecuador']
dataset_read = pd.DataFrame(dataset_read.iloc[0].loc['1960':])


In [166]:
dataset_read.reset_index(inplace=True)
dataset_read.columns = ['year','pib_value']

In [167]:
alt.Chart(df.melt('year')).mark_line().encode(
x='year',
y=alt.Y('value',title='PIB'),
color='variable'
).properties(title='PIB Ecuador')

alt.Chart(...)

In [160]:
dataset_read['change']=dataset_read['pib_value'].diff()
dataset_read['change']=(dataset_read['change']/dataset_read['pib_value']) *100
dataset_read=df.fillna(0)


In [162]:
alt.Chart(dataset_read.drop('pib_value',axis=1).melt('year')).mark_line().encode(
x='year',
y=alt.Y('value',title='porcentaje de cambio %')
).properties(title='PIB en Ecuador')

alt.Chart(...)

In [183]:
data=pd.DataFrame(df['pib_value'].diff())
data=data.fillna(0.00)
dict_n_vs_p = {'positiva':data.query('pib_value>0').sum()[0],'negativa':np.abs(data.query('pib_value<0').sum()[0])}
df_pn = pd.DataFrame(dict_n_vs_p,index=[0]).T
df_pn.reset_index(inplace=True)
df_pn.columns = ['description','value']

In [184]:
alt.Chart(df_pn).mark_bar(size=30).encode(
x='description',
y=alt.Y('value',title='PIB '),
color='description'
).properties(width=alt.Step(150),title='Afeccion del COVID en el PIB')

alt.Chart(...)

In [90]:
def _obtain_feature_target(df,feature_columns,target_columns):
    features = df[feature_columns]
    target= df[target_columns]
    return features,target


In [91]:
features,target = _obtain_feature_target(df,['year'],'pib_value')

In [92]:
model_linear = LinearRegression()
model_linear = model_linear.fit(features,target)
print("Score linear model = {}".format(model_linear.score(features,target)))

Score linear model = 0.6709336527848547


In [192]:
predict_linear = pd.DataFrame({'year':features.values.reshape(-1),'real':target.values,'predict':model_linear.predict(features)})

In [197]:
alt.Chart(predict_linear.melt('year')).mark_line().encode(
x='year',
y='value',
color='variable'
).properties(title='Prediccion Lineal')

alt.Chart(...)

In [28]:
model_poly = Pipeline([('poly', PolynomialFeatures(degree=3)),
('linear', LinearRegression(fit_intercept=True))])
model_poly = model_poly.fit(features,target)
print("Score polinomyal model = {}".format(model_poly.score(features,target)))

Score polinomyal model = 0.7134725577933035


In [30]:
predict_polinomyal = pd.DataFrame({'year':features.values.reshape(-1),'real':target.values,'predict':model_poly.predict(features)})

In [198]:
alt.Chart(predict_polinomyal.melt('year')).mark_line().encode(
x='year',
y='value',
color='variable'
).properties(title='prediccion polinomial')

alt.Chart(...)

In [199]:
predictions=pd.DataFrame({'year':[i for i in range(2020,2026)]})
df_predict = pd.DataFrame({'year':['{}'.format(i) for i in range(2020,2026)],
    'Lineal':model_linear.predict(predictions),
    'Polinomial':model_poly.predict(predictions)})

In [200]:
df_predict

,year,Lineal,Polinomial
0,2020,"4,876.90","5,897.90"
1,2021,"4,962.86","6,116.56"
2,2022,"5,048.81","6,342.18"
3,2023,"5,134.76","6,574.90"
4,2024,"5,220.71","6,814.86"
5,2025,"5,306.66","7,062.20"
